In [1]:
import numpy as np
import pandas as pd

In [82]:
raw_data=pd.read_csv("../data/cleaned_data.csv")
tendecy_one_hot=pd.get_dummies(raw_data.loc[:,"Tendency"])
raw_data=pd.concat([raw_data,tendecy_one_hot],axis=1)
raw_data.drop(['Tendency','DR'],axis=1, inplace=True)
raw_data.columns

Index(['FileName', 'Date', 'SegFile', 'b', 'e', 'LBE', 'LB', 'AC', 'FM', 'UC',
       'ASTV', 'MSTV', 'ALTV', 'MLTV', 'DL', 'DS', 'DP', 'Width', 'Min', 'Max',
       'Nmax', 'Nzeros', 'Mode', 'Mean', 'Median', 'Variance', 'NSP',
       'left_assymetric', 'right_symmetric', 'symmetric'],
      dtype='object')

In [108]:
continuous_attributes=raw_data.iloc[:,3:26]
categorical_attributes=raw_data.iloc[:,26:]
continuous_attributes=(continuous_attributes-continuous_attributes.min())/(continuous_attributes.max()-continuous_attributes.min())
dataset=pd.concat([continuous_attributes,categorical_attributes.iloc[:,1:],categorical_attributes.iloc[:,0]],axis=1)
dataset.to_csv("../data/cleaned_normalized_data.csv",index=False)

In [148]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
import math

In [150]:
dataset_array=dataset.values
no_of_instances=len(dataset_array)
split=math.floor(0.75*no_of_instances)
data_x=dataset_array[:,:-1]
data_y=dataset_array[:,-1]
X_train=dataset_array[:split,:-1]
y_train=dataset_array[:split,-1]
X_test=dataset_array[split:,:-1]
y_test=dataset_array[split:,-1]

1594

In [151]:
classifier = svm.SVC(kernel='rbf', C = 1.0, verbose=True)

classifier.fit(X_train,y_train)
y_pred=classifier.predict(X_test)
confusion_mtx = confusion_matrix(y_test, y_pred) 
print("\n",confusion_mtx)
target_names = ['normal', 'suspect', 'pathological']
print(classification_report(y_test, y_pred, target_names=target_names))
print("accuracy= ",(confusion_mtx.diagonal().sum()/confusion_mtx.sum())*100)



[LibSVM]
 [[411  25   0]
 [  6   3   0]
 [ 87   0   0]]
              precision    recall  f1-score   support

      normal       0.82      0.94      0.87       436
     suspect       0.11      0.33      0.16         9
pathological       0.00      0.00      0.00        87

 avg / total       0.67      0.78      0.72       532

accuracy=  77.8195488722


/home/kenny/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
